In [15]:
from dotenv import load_dotenv
import os
load_dotenv("/media/uberdev/ddrv/gitFolders/codeai_fusion/.env")
from dspy import OpenAI

In [2]:
from openinference.instrumentation.dspy import DSPyInstrumentor
# instruments the internal calls in DSPy library
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
# help to get the span of http requests to the APIs
from opentelemetry.sdk import trace as trace_sdk
#processes the data collected from the spans
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from openinference.semconv.resource import ResourceAttributes
import phoenix as px

INFO:phoenix.config:📋 Ensuring phoenix working directory: /home/uberdev/.phoenix
INFO:phoenix.inferences.inferences:Dataset: phoenix_inferences_77bd4f9a-0006-47b3-af62-c2495b5d155b initialized
/media/uberdev/ddrv/telemetenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# before executing this cell, ensure phoenix server is running with the postgres database
# will get a warning to start a local server, if not running.
endpoint = "http://127.0.0.1:6006/v1/traces"
client = px.Client(endpoint=endpoint)
# https://docs.arize.com/phoenix/tracing/how-to-tracing/trace-a-deployed-app
resource = Resource(attributes={
    ResourceAttributes.PROJECT_NAME: 'instrumenting-dspy'
})

INFO:httpx:HTTP Request: GET http://127.0.0.1:6006/v1/traces/arize_phoenix_version "HTTP/1.1 200 OK"


In [11]:
tracer_provider = trace_sdk.TracerProvider(resource=resource)
span_otlp_exporter = OTLPSpanExporter(endpoint=endpoint)

In [12]:
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter=span_otlp_exporter))

trace_api.set_tracer_provider(tracer_provider=tracer_provider)

DSPyInstrumentor().instrument(skip_dep_check=True) # here where DSPy is instrumented

In [13]:
# Need to check if the dspy is writing logs
import os
llm = OpenAI(model='gpt-4o-mini',
             api_key=os.environ['OPENAI_API_KEY'],
             max_tokens=2000)

In [14]:
llm("Let's see if traces are logging to the console")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


['To check if traces are logging to the console, you can follow these steps depending on the environment you\'re working in:\n\n### For JavaScript in a Web Browser:\n1. **Open Developer Tools**: Right-click on the webpage and select "Inspect" or press `F12`.\n2. **Go to the Console Tab**: Click on the "Console" tab in the Developer Tools.\n3. **Check for Logs**: Look for any `console.log()`, `console.warn()`, or `console.error()` messages that your code may be outputting.\n\n### For Node.js:\n1. **Run Your Application**: Start your Node.js application in the terminal.\n2. **Check the Terminal Output**: Look for any console logs that are printed to the terminal. You can use `console.log()` in your code to log messages.\n\n### For Python:\n1. **Run Your Script**: Execute your Python script in the terminal or command prompt.\n2. **Check the Output**: Any `print()` statements will output to the console.\n\n### For Other Languages:\n- Ensure that you are using the appropriate logging functi